## building an agent that interacts with a search engine 
- ask agent questions 
- watch agent call the search tool 
- have conversations with the agent

[python.langchain tutorial](https://python.langchain.com/docs/tutorials/agents/)

In [1]:
import getpass
import os
from dotenv import load_dotenv

# os.environ["LANGMSITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

# loads variables from .env
load_dotenv()

tracing = os.getenv("LANGSMITH_TRACING", "false")
api_key = os.getenv("LANGSMITH_API_KEY")

print("Tracing:", tracing)
print("API Key is set:", api_key is not None)

tav_key = os.getenv("TAVILY_API_KEY")
print("Tavily API Key is set:", tav_key is not None)
# os.environ["TAVLIY_API_KEY"] = getpass.getpass()

gemini_api = os.getenv("GOOGLE_API_KEY")
print("Google Gemini API Key is set")

print("GOOGLE_API_KEY exists?", "GOOGLE_API_KEY" in os.environ)

Tracing: true
API Key is set: True
Tavily API Key is set: True
Google Gemini API Key is set
GOOGLE_API_KEY exists? True


**note: Tavily researcher plan gives 1000 API credits (monthly) for free, no credit card required**

In [3]:
# we need to create the tools we want to use
# main tool of choice will be Tavily (a search engine)
# we can use the dedicated langchain-tavily integration
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF?")
print(search_results)
# if we want, we can create other tools 
# once we have all the tools we want, we can put them in a list that we will reference later
tools = [search]

{'query': 'What is the weather in SF?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco, CA', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751377166, 'localtime': '2025-07-01 06:39'}, 'current': {'last_updated_epoch': 1751376600, 'last_updated': '2025-07-01 06:30', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 6.0, 'wind_kph': 9.7, 'wind_degree': 236, 'wind_dir': 'WSW', 'pressure_mb': 1012.0, 'pressure_in': 29.87, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 100, 'feelslike_c': 12.0, 'feelslike_f': 53.6, 'windchill_c': 11.4, 'windchill_f': 52.5, 'heatindex_c': 12.2, 'heatindex_f': 53.9, 'dewpoint_c': 12.0, '

In [4]:
from langchain.chat_models import init_chat_model
from langchain_google_genai import ChatGoogleGenerativeAI
# model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7
)

# response = llm.invoke("Tell me a fun fact about octopuses.")
# print(response.content)

query = "Hi"
response = llm.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help you today?'

In [5]:
# import relevant functionality 
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

packages i need to install in virtual environment
```
pip install langchain
pip insall langgraph
pip install langchain-community tavily-python
pip install -U langchain-anthropic
pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite
pip install -U langchain-google-genai
```

confirmed i have the correct packages:
```
pip show langgraph
pip show langchain-community
pip show tavily-python
```

<span style="color: yellow;">ensure you have a Tavily API key set as TAVILY_API_KEY in the environment</span>

In [15]:
# create the agent 
memory = MemorySaver()
# i think this is free ? 
model = init_chat_model("anthropic:claude-3-5-sonnet-latest")
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [7]:
# use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user", 
    "content": "Hi, I'm Bob and I live in SF.",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

NameError: name 'agent_executor' is not defined

In [9]:
# use .bind_tools to give the language model knowledge of tools
model_with_tools = llm.bind_tools(tools)

In [10]:
# we can now call the model --> look @ both the content field + the tool_calls field
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hi, how can I help you today?

Tool calls: []


In [11]:
# calling model with input that would expect a tool to be called 
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'Weather in San Francisco'}, 'id': '0a7b85f1-02cb-4e27-b5fe-56722b7dbda0', 'type': 'tool_call'}]


^ no text content, but there is a tool call --> it wnats us to call the Tavily Search tool 
- it isn't calling the tool yet, it is just telling us to 
- in order to actually call the tool, we have to create an agent

---
## create the agent
now that we have defined the tools + the LLM, we can create the agent
- using LangGraph to construct the agent 
- high level interface backed by low level + controllable API (allows user to easily modify the agnet logic)
- time to initialize the agent with the LLM + tools 

In [12]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(llm, tools)

## run the agent
we will now run the agent with a few queries 
- all stateless queries (won't remember previous interactions)
- the agent will return the final state @ the end of the interaction 
    - includes any inputs
    - later on --> see how to get only outputs

In [13]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hi, how can I help you today?


in order to see exactly what is happening under the hood + ensure it's not calling a tool, we can take a look @ the LangSmith trace

In [14]:
# example where it should be invoking the tool 
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (285dbb56-1dca-49b4-96bf-4dea889f9ae7)
 Call ID: 285dbb56-1dca-49b4-96bf-4dea889f9ae7
  Args:
    query: Weather in San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "Weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751379494, 'localtime': '2025-07-01 07:18'}, 'current': {'last_updated_epoch': 1751379300, 'last_updated': '2025-07-01 07:15', 'temp_c': 12.8, 'temp_f': 55.0, 'is_d

## streaming messages
we've seen how the agent can be called with ```.invoke``` to get a final response. if the agent executes multiple steps, this may take a while. to show intermediate progress, we cna stream back messages as they occur.

In [15]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (75de1bc0-b5c2-4338-9bd4-b974f8312e4f)
 Call ID: 75de1bc0-b5c2-4338-9bd4-b974f8312e4f
  Args:
    query: Weather in San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "Weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751379494, 'localtime': '2025-07-01 07:18'}, 'current': {'last_updated_epoch': 1751379300, 'last_updated': '2025-07-01 07:15', 'temp_c': 12.8, 'temp_f': 55.0, 'is_d